In [65]:
import cv2
import os
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
import time

In [66]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]
def load_image(image_path):
    
    try:
        img = cv2.imread(image_path)
        return img
    except Exception as e:
        print("error read image: ", image_path, " ", e)
        return None

# cheking image and return true/false
def is_image_file(file_path):
    
    extensions =(".jpg", ".ipeg", ".png", ".gif", ".bmp")
    return file_path.lower().endswith(extensions)

def get_image_list(folder_path):
    image_list = []
    image_name_list=[]
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        filenames = sorted(os.listdir(folder_path), key=natural_sort_key)
        for filename in filenames:
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path) and is_image_file(file_path):
                img = load_image(file_path)
                image_list.append(img)
                image_name_list.append(filename)
    return image_list,image_name_list

In [67]:
def gray_all(list_image):
    new_image_list=[]
    for img in list_image:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        new_image_list.append(img)
    return new_image_list
def compute_gist(list_image, orientations=8, blocks=4):
    starttime=time.time()
    list_descriptor=[]
    for img in tqdm(list_image):
        height, width = img.shape

        # Define cell size
        cell_size = min(height, width) // blocks

        # Calculate gradient magnitude and orientation
        gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
        gradient_magnitude = np.sqrt(gx**2 + gy**2)
        gradient_orientation = np.arctan2(gy, gx) * (180 / np.pi) + 180
        gradient_orientation_bins = np.floor(gradient_orientation / (360 / orientations)).astype(int)

        # Compute GIST descriptor
        descriptor = np.zeros(orientations * blocks * blocks)

        for i in range(blocks):
            for j in range(blocks):
                cell_hist = np.zeros(orientations)
                for ii in range(cell_size):
                    for jj in range(cell_size):
                        x = i * cell_size + ii
                        y = j * cell_size + jj
                        bin_idx = gradient_orientation_bins[x, y] % orientations
                        cell_hist[bin_idx] += gradient_magnitude[x, y]
                descriptor[(i * blocks + j) * orientations: (i * blocks + j + 1) * orientations] = cell_hist

        # Normalize descriptor
        descriptor /= np.sum(descriptor)
        list_descriptor.append(descriptor)
    extractiontime=time.time()-starttime
    return list_descriptor,extractiontime
def vectorT(descriptors):
    new_descriptors=[]
    for matrix in tqdm(descriptors):
        matrix=matrix.flatten()
        matrix=matrix.reshape(1,-1)
        matrix=pd.DataFrame(matrix)
        new_descriptors.append(matrix)
    return new_descriptors

In [68]:
folder_path='D:\mon_hoc\S4 AIL303M\Rice_photo\Xi23 fol\Xi23\Xi-23'
list_image,list_image_name=get_image_list(folder_path)
list_image=gray_all(list_image)
descriptors,time1=compute_gist(list_image)

100%|██████████| 2227/2227 [00:06<00:00, 323.57it/s]


In [69]:
folder_path1='D:\mon_hoc\S4 AIL303M\Rice_photo\Xi23 fol\Xi23\\Negative'
list_image1,list_image_name1=get_image_list(folder_path1)
list_image1=gray_all(list_image1)
descriptors1,time2=compute_gist(list_image1)

100%|██████████| 1918/1918 [00:06<00:00, 317.66it/s]


In [70]:
# extractiontime=time1+time2
# print("Extraction time:",extractiontime)

In [71]:
descriptors=vectorT(descriptors)
descriptors1=vectorT(descriptors1)

100%|██████████| 1918/1918 [00:00<00:00, 43040.37it/s]


In [72]:
df=pd.concat(descriptors)
df.reset_index(drop=True, inplace=True)

In [73]:
df1=pd.concat(descriptors1)
df1.reset_index(drop=True, inplace=True)

In [74]:
df["Name"]=list_image_name
df1["Name"]=list_image_name1

In [75]:
df = df.iloc[:, [-1] + list(range(df.shape[1] - 1))]
df1 = df1.iloc[:, [-1] + list(range(df1.shape[1] - 1))]

In [76]:
df

,Name,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,DSC6774_idx1.png,0.016552,0.052570,0.004906,0.001122,0.000669,0.001763,0.001239,0.013235,0.001442,...,0.041991,0.004833,0.001678,0.003685,0.002250,0.001415,0.001445,0.005268,0.042538,0.003080
1,DSC6774_idx2.png,0.001267,0.000715,0.001477,0.001128,0.001500,0.000683,0.001804,0.001534,0.001779,...,0.046268,0.006289,0.000636,0.002287,0.002428,0.001896,0.002067,0.027945,0.031925,0.003158
2,DSC6774_idx3.png,0.001214,0.000833,0.001287,0.001563,0.001448,0.001252,0.001702,0.001610,0.001901,...,0.040539,0.003891,0.001592,0.006556,0.010997,0.000808,0.001291,0.022937,0.030444,0.001714
3,DSC6774_idx4.png,0.001746,0.001395,0.001620,0.001032,0.001289,0.001402,0.001070,0.000872,0.002246,...,0.048408,0.002868,0.001431,0.001408,0.001790,0.002003,0.002247,0.006657,0.046163,0.004505
4,DSC6774_idx5.png,0.010898,0.021051,0.016498,0.002942,0.002411,0.003661,0.002200,0.001508,0.002668,...,0.049544,0.006849,0.001624,0.003127,0.004154,0.004201,0.003289,0.009250,0.036328,0.001914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,DSC6833_idx75.png,0.021560,0.023893,0.023359,0.002049,0.002839,0.004542,0.006763,0.005113,0.007365,...,0.049803,0.002686,0.002751,0.003567,0.004186,0.003135,0.001961,0.015611,0.039654,0.002960
2223,DSC6833_idx76.png,0.001595,0.015008,0.002962,0.000858,0.000769,0.001515,0.001703,0.000782,0.007954,...,0.076791,0.010303,0.002517,0.009399,0.025004,0.007561,0.005853,0.027768,0.046323,0.000875
2224,DSC6833_idx77.png,0.001567,0.001191,0.001266,0.001191,0.001719,0.001019,0.000814,0.000885,0.002293,...,0.050187,0.002936,0.001563,0.000874,0.000634,0.000952,0.002377,0.016873,0.031731,0.001890
2225,DSC6833_idx78.png,0.017046,0.037518,0.012520,0.002490,0.003855,0.006410,0.005559,0.004447,0.003880,...,0.047893,0.003757,0.001495,0.001196,0.001124,0.001572,0.001724,0.013657,0.038764,0.002832


In [77]:
df1

,Name,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,DSC6527_idx43.png,0.000578,0.001216,0.000820,0.000840,0.001092,0.000237,0.000983,0.000533,0.001249,...,0.067337,0.002973,0.000101,0.002036,0.004409,0.006759,0.003028,0.018941,0.053614,0.001380
1,DSC6528_idx1.png,0.000848,0.000993,0.001338,0.000888,0.001132,0.001145,0.001260,0.000942,0.001841,...,0.039998,0.004290,0.001745,0.015301,0.015968,0.001630,0.002401,0.029418,0.030655,0.001989
2,DSC6528_idx2.png,0.005559,0.025687,0.021104,0.002244,0.001278,0.001111,0.015085,0.006716,0.005714,...,0.044548,0.006484,0.001762,0.003062,0.006851,0.002878,0.002365,0.004719,0.061156,0.004353
3,DSC6528_idx3.png,0.000941,0.001265,0.001411,0.000822,0.001073,0.001345,0.001544,0.001296,0.005730,...,0.039849,0.002570,0.001192,0.001063,0.001440,0.000971,0.003134,0.024040,0.033701,0.002119
4,DSC6528_idx4.png,0.002042,0.001888,0.001598,0.001173,0.001690,0.001866,0.001832,0.001667,0.001561,...,0.068346,0.002032,0.001501,0.000560,0.000574,0.001108,0.002896,0.012391,0.056965,0.005063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1913,DSC7432_idx35.png,0.001166,0.001107,0.001305,0.000785,0.001103,0.000865,0.000826,0.000732,0.001085,...,0.053364,0.004213,0.001075,0.001204,0.001479,0.001835,0.001651,0.032224,0.037478,0.003910
1914,DSC7432_idx36.png,0.001165,0.001258,0.001868,0.000843,0.000942,0.001259,0.001967,0.001216,0.001851,...,0.059962,0.002924,0.001009,0.001338,0.001435,0.001702,0.000985,0.017757,0.048368,0.003297
1915,DSC7432_idx37.png,0.002488,0.004939,0.001489,0.001460,0.001753,0.002000,0.001940,0.001407,0.002513,...,0.051154,0.001924,0.000834,0.004255,0.005590,0.002244,0.001463,0.006920,0.051157,0.003949
1916,DSC7432_idx38.png,0.001404,0.001556,0.001470,0.001284,0.001104,0.000688,0.001954,0.001097,0.001867,...,0.036604,0.001111,0.001339,0.003078,0.007250,0.001213,0.001660,0.010963,0.050696,0.006357


In [78]:
new_df=pd.concat([df,df1],axis=0)

In [79]:
new_df

,Name,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,DSC6774_idx1.png,0.016552,0.052570,0.004906,0.001122,0.000669,0.001763,0.001239,0.013235,0.001442,...,0.041991,0.004833,0.001678,0.003685,0.002250,0.001415,0.001445,0.005268,0.042538,0.003080
1,DSC6774_idx2.png,0.001267,0.000715,0.001477,0.001128,0.001500,0.000683,0.001804,0.001534,0.001779,...,0.046268,0.006289,0.000636,0.002287,0.002428,0.001896,0.002067,0.027945,0.031925,0.003158
2,DSC6774_idx3.png,0.001214,0.000833,0.001287,0.001563,0.001448,0.001252,0.001702,0.001610,0.001901,...,0.040539,0.003891,0.001592,0.006556,0.010997,0.000808,0.001291,0.022937,0.030444,0.001714
3,DSC6774_idx4.png,0.001746,0.001395,0.001620,0.001032,0.001289,0.001402,0.001070,0.000872,0.002246,...,0.048408,0.002868,0.001431,0.001408,0.001790,0.002003,0.002247,0.006657,0.046163,0.004505
4,DSC6774_idx5.png,0.010898,0.021051,0.016498,0.002942,0.002411,0.003661,0.002200,0.001508,0.002668,...,0.049544,0.006849,0.001624,0.003127,0.004154,0.004201,0.003289,0.009250,0.036328,0.001914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1913,DSC7432_idx35.png,0.001166,0.001107,0.001305,0.000785,0.001103,0.000865,0.000826,0.000732,0.001085,...,0.053364,0.004213,0.001075,0.001204,0.001479,0.001835,0.001651,0.032224,0.037478,0.003910
1914,DSC7432_idx36.png,0.001165,0.001258,0.001868,0.000843,0.000942,0.001259,0.001967,0.001216,0.001851,...,0.059962,0.002924,0.001009,0.001338,0.001435,0.001702,0.000985,0.017757,0.048368,0.003297
1915,DSC7432_idx37.png,0.002488,0.004939,0.001489,0.001460,0.001753,0.002000,0.001940,0.001407,0.002513,...,0.051154,0.001924,0.000834,0.004255,0.005590,0.002244,0.001463,0.006920,0.051157,0.003949
1916,DSC7432_idx38.png,0.001404,0.001556,0.001470,0.001284,0.001104,0.000688,0.001954,0.001097,0.001867,...,0.036604,0.001111,0.001339,0.003078,0.007250,0.001213,0.001660,0.010963,0.050696,0.006357


In [80]:
new_df.to_csv("Xi-23GIST.csv")